In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('taxi_dataset.csv')

In [137]:
taxiDB.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


<dl>
<dt> Описание колонок:
<dd>id - ID поездки </dd>
<dd>vendor_id - ID компании, осуществляющей перевозку </dd>
<dd>pickup_datetime - Таймкод начала поездки</dd>
<dd>dropoff_datetime - Таймкод конца поездки </dd>
<dd>passenger_count - Количество пассажиров </dd>
<dd>pickup_longitude - Долгота точки, в которой началась поездка </dd>
<dd>pickup_latitude - Широта точки, в которой началась поездка </dd>
<dd>dropoff_longitude - Долгота точки, в которой закончилась поездка </dd>
<dd>dropoff_latitude - Широта точки, в которой закончилась поездка </dd>
<dd>store_and_fwd_flag - Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером </dd>
</dl>

**Целевая переменная - длительность поездки.**

In [138]:
taxiDB.pickup_datetime = pd.to_datetime(taxiDB.pickup_datetime)
taxiDB.dropoff_datetime = pd.to_datetime(taxiDB.dropoff_datetime)
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime'] - taxiDB['pickup_datetime']).dt.seconds

In [139]:
taxiDB.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124


In [152]:
taxiDB = taxiDB.drop('dropoff_datetime', axis=1)

In [140]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1
taxiDB.loc[(taxiDB['store_and_fwd_flag'] == 'N'), 'save_in_memory'] = 0
taxiDB.save_in_memory = taxiDB.save_in_memory.fillna(1)
taxiDB = taxiDB.drop('store_and_fwd_flag', axis=1)

In [141]:
taxiDB.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,save_in_memory
0,id2875421,1,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,455,0.0
1,id2377394,0,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,663,0.0
2,id3858529,1,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,2124,0.0


Во-вторых, можем использовать долготу и широту точек старта/завершения поездки, чтобы примерно оценить расстояние между 2 точками.

Сами по себе они, как самостоятельные вещественные признаки, вряд ли способны хорошо объяснять длительность поездки.

Базовая идея состоит в том, чтобы посчитать разность долгот и широт соответственно, то есть:

$$
\delta_{long} = \text{dropoff_longitude} - \text{pickup_longitude}
$$

$$
\delta_{lat} =  \text{dropoff_latitude} - \text{pickup_latitude}
$$

А потом вычислить географическое расстояние между 2 точками по теореме Пифагора:

$$
R = \sqrt{\delta^2_{long} + \delta^2_{lat}}
$$

Мы реализуем данную задумку и вычислим такую вещественную колонку **R**, что, в целом, является хорошим тоном при работе с координатами точек.

Только для начала нужно некоторым образом перевести долготу и широту в километры, обеспечив равенство их мер измерения. Потому что, вообще говоря, *градусная мера* широт и долгот имеет неодинаковую шкалу перевода в километры. Так, если пропустить данную деталь, расстояние **R** будет вычислено неверно, ведь катеты тогда будут иметь разную размерность.

В целом, перевод из долгот и широт в расстояние поездки позволяет нам в будущем проверить зависимость **длительности поездки от километража**, и объяснить ее будет куда проще, чем аналогичную между таргетом и изначальными признаками

<a href="https://www.datafix.com.au/BASHing/2018-11-07.html"> Маленькая статья про перевод разницы градусов долгот/широт в километры</a>

**Начнем переводить каждую долготу в некоторое относительно километровое выражение**

Соберем список из всех широт (как точек старта, так и конца).

In [142]:
allLat  = list(taxiDB['pickup_latitude']) + list(taxiDB['dropoff_latitude'])

In [143]:
medianLat  = sorted(allLat)[int(len(allLat)/2)]

In [144]:
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)

In [145]:
taxiDB.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,save_in_memory
0,id2875421,1,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,1.516008,-73.964630,1.256121,455,0.0
1,id2377394,0,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,-1.753813,-73.999481,-2.578912,663,0.0
2,id3858529,1,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,1.070973,-74.005333,-4.923841,2124,0.0
3,id3504673,1,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,-3.823568,-74.012268,-5.298809,429,0.0
4,id2181028,1,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,4.329328,-73.972923,3.139453,435,0.0


Итого, для **latitude** колонок получили следующие выражения:

*На сколько примерно километров севернее или южнее (в зависимости от знака) точка находится относительно средней широты*

In [146]:
allLong = list(taxiDB['pickup_longitude']) + list(taxiDB['dropoff_longitude'])

medianLong  = sorted(allLong)[int(len(allLong)/2)]

longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32

In [147]:
taxiDB['pickup_longitude']   = longMultiplier  * (taxiDB['pickup_longitude']   - medianLong)
taxiDB['dropoff_longitude']   = longMultiplier  * (taxiDB['dropoff_longitude']  - medianLong)

taxiDB.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,save_in_memory
0,id2875421,1,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-0.110015,1.516008,1.367786,1.256121,455,0.0
1,id2377394,0,2016-06-12 00:43:35,2016-06-12 00:54:38,1,0.036672,-1.753813,-1.571088,-2.578912,663,0.0
2,id3858529,1,2016-01-19 11:35:24,2016-01-19 12:10:48,1,0.153763,1.070973,-2.064547,-4.923841,2124,0.0
3,id3504673,1,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-2.461500,-3.823568,-2.649362,-5.298809,429,0.0
4,id2181028,1,2016-03-26 13:30:55,2016-03-26 13:38:10,1,0.657515,4.329328,0.668452,3.139453,435,0.0


Медианы позволяют нам во время вычисления расстояния преобразовать изначальные longtitude/latitude колонки в "отдаленности точек старта/конца поездок" от медианных точек. Есть подозрение, что медианная для поездок точка города - это, на практике, точка скопления вечерних пробок. 
Может быть вполне важно знать, насколько далеко от такого эпицентра ужаса мы начинаем и заканчиваем поездку (насколько севернее/южнее/...) и выделить поверх этой информации дополнительные признаки.<br>
В домашнем задании это использоваться не будет, но это ещё один пример, как можно работать с признаками.

Наконец, вычислим географическое расстояние **distance_km**:

In [148]:
taxiDB['distance_km'] = \
         np.sqrt( \
             (taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude'])**2 + \
             (taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude'])**2 \
         )

In [149]:
taxiDB.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,save_in_memory,distance_km
0,id2875421,1,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-0.110015,1.516008,1.367786,1.256121,455,0.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,2016-06-12 00:54:38,1,0.036672,-1.753813,-1.571088,-2.578912,663,0.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,2016-01-19 12:10:48,1,0.153763,1.070973,-2.064547,-4.923841,2124,0.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-2.461500,-3.823568,-2.649362,-5.298809,429,0.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,2016-03-26 13:38:10,1,0.657515,4.329328,0.668452,3.139453,435,0.0,1.189925


In [150]:
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude'], axis=1)

In [153]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,trip_duration,save_in_memory,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,455,0.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,663,0.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,2124,0.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,1,429,0.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,1,435,0.0,1.189925


In [154]:
taxiDB.passenger_count.value_counts()

passenger_count
1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: count, dtype: int64

Какой это признак, на ваш взгляд: вещественный, категориальный, порядковый? 

С одной стороны, можно воспринимать его как обычный вещественный признак. Ведь само по себе количество пассажиров (без дополнительной обработки) - это некоторое число, которое может принимать большое количество различных значений.

С другой стороны, мы с Вами наверняка знаем, что количество пассажиров от поездки к поездке ограничено. Вряд ли если к нам придут новые данные, мы увидим числа бОльшие, чем у нас в датасете. Тогда рассуждаем следующим образом: раз множество значений признака ограничено, то он категориальный (или, в данном случае, даже порядковый! Ведь у нас могут быть какие-то логичные предположения о том, что количество пассажиров может влиять на модель машины и, соответственно, скорость ее передвижения и скорость поездки!)

Предлагаю Вам реализовать прием с **Mean-target encoding'ом**, как в практическом занятии. Замените колонку **passenger_count** колонкой **category_encoded**.

In [156]:
taxiDB['passenger_count'] = taxiDB['passenger_count'].map(taxiDB
                                                          .groupby('passenger_count')['trip_duration']
                                                          .mean())

In [157]:
taxiDB = taxiDB.set_index('id')

In [158]:
taxiDB.head()

,vendor_id,pickup_datetime,passenger_count,trip_duration,save_in_memory,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,923.126885,455,0.0,1.500479
id2377394,0,2016-06-12 00:43:35,923.126885,663,0.0,1.807119
id3858529,1,2016-01-19 11:35:24,923.126885,2124,0.0,6.392080
id3504673,1,2016-04-06 19:32:31,923.126885,429,0.0,1.487155
id2181028,1,2016-03-26 13:30:55,923.126885,435,0.0,1.189925


In [167]:
X = taxiDB.drop('trip_duration', axis=1).drop('pickup_datetime', axis=1)
Y = taxiDB['trip_duration']

In [169]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, Y)

X['Prediction'] = model.predict(X)

X.head()

,vendor_id,passenger_count,save_in_memory,distance_km,Prediction
id,,,,,
id2875421,1,923.126885,0.0,1.500479,825.241538
id2377394,0,923.126885,0.0,1.807119,646.849337
id3858529,1,923.126885,0.0,6.392080,1372.699858
id3504673,1,923.126885,0.0,1.487155,823.750316
id2181028,1,923.126885,0.0,1.189925,790.485006


In [161]:
Y.head()

id
id2875421     455
id2377394     663
id3858529    2124
id3504673     429
id2181028     435
Name: trip_duration, dtype: int32